In [1]:
#Colab read data from google drive
import os
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/MyDrive/Pornpan(Eye)')

Mounted at /content/drive/


In [2]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders 

## Select features

🎯 The impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using `orders` training_set,  run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [3]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [4]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,30.174037


In [5]:
#Select in a list which features you want to use
# select_orders = ["wait_time","delay_vs_expected","dim_is_five_star","dim_is_one_star"]
model1 = smf.logit(formula='dim_is_one_star ~  wait_time + delay_vs_expected', data=orders).fit();
model1.params

#smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().summary()

Optimization terminated successfully.
         Current function value: 0.283196
         Iterations 7


Intercept           -3.193266
wait_time            0.059117
delay_vs_expected    0.072307
dtype: float64

👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

In [6]:


# wait_time,delay_vs_expected,good/bad reviews

# multivariate logistic regressions

model2 = smf.logit(formula='dim_is_five_star ~ wait_time + delay_vs_expected ', data=orders).fit();
model2.params



Optimization terminated successfully.
         Current function value: 0.642678
         Iterations 7


Intercept            0.999549
wait_time           -0.046604
delay_vs_expected   -0.105796
dtype: float64

🕵🏻 Check the `multi-colinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    


⚖️ Standardizing:

In [1]:

#mpg_scaled = mpg.copy()
    
#mpg_scaled

feature = ["wait_time","delay_vs_expected","dim_is_five_star","dim_is_one_star"]

feature_orders = orders.copy()

for i in feature:
    mu = orders[i].mean()
    sigma = orders[i].std()
    feature_orders[i] = orders[i].map(lambda x: (x-mu)/sigma)
feature_orders

NameError: ignored

In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
# compute VIF factor for feature index 0



# from order.stats.outliers_influence import variance_inflation_factor as vif
# vif(feature_orders.values, 0)


👉 Run your VIF Analysis to analyze the potential multicolinearities:

In [9]:

df = pd.DataFrame()

df["feature"] = feature_orders[feature].columns

df["vif_index"] = [vif(feature_orders[feature].values, i) for i in range(feature_orders[feature].shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

,feature,vif_index
0,wait_time,2.06
1,delay_vs_expected,2.00
3,dim_is_one_star,1.27
2,dim_is_five_star,1.21


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [10]:

#logit_one to predict dim_is_one_star
logit_one = smf.logit(formula='dim_is_one_star ~  1', data=orders).fit();
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.319891
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95871
Method:                           MLE   Df Model:                            0
Date:                Wed, 24 Aug 2022   Pseudo R-squ.:               7.886e-11
Time:                        09:47:10   Log-Likelihood:                -30669.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2236      0.011   -204.375      0.000      -2.245      -2.202
==============================================================================
"""

`Logit 5️⃣`

In [11]:

#logit_five to predict dim_is_five_star
logit_five = smf.logit(formula='dim_is_five_star ~  1', data=orders).fit();
logit_five.summary()


Optimization terminated successfully.
         Current function value: 0.676080
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95871
Method:                           MLE   Df Model:                            0
Date:                Wed, 24 Aug 2022   Pseudo R-squ.:               5.367e-11
Time:                        09:47:15   Log-Likelihood:                -64817.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3727      0.007     56.713      0.000       0.360       0.386
==============================================================================
"""

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [12]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = [a]

<

> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...


